In [1]:
import string
import re
import codecs
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import feature_extraction
from sklearn import linear_model
from sklearn import pipeline
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import accuracy_score
from keras.models import Model
from keras import optimizers
from sklearn.model_selection import cross_val_score

In [2]:
pun_df = pd.read_csv("Punjabi.csv",header=None,names=["Punjabi"])
pun_df.head()

,Punjabi
0,ਉਨ੍ਹਾਂ ਨੇ ਸਾਰੇ ਤਜ਼ਰਬੇ ਵੱਖਰੀ ਕਿਸਮ ਦੇ ਕੀਤੇ ਹਨ
1,ਸ਼ਹਿਰਾਂ ਗਲੀਆਂ ਵਿਚ ਸਾਰੇ ਲਿਖਵਾ ਦਿਆਂ
2,ਆਊਟਡੋਰਸ ਚ ਪਏ ਟੋਇਆਂ ਦੀ ਰੀਪੇਅਰ ਕਰਵਾਓ
3,ਜੰਗ ਵਿਆਂ ਵਿੱਚ ਲੜੀ ਗਈ
4,ਧਰਮ ਦੇ ਨਾਂ ਤੇ ਦੇਸ਼ ਦੀ ਵੰਡ


In [3]:
eng_df = pd.read_csv("English.csv",header=None,names=["English"])
eng_df.head()

,English
0,Professor Floyd pointed out that mass lesions...
1,The Norsemen considered the rainbow as a brid...
2,She can scoop these things into three red bag...
3,Islay is one of five whisky distilling locali...
4,Messages from Molly and Mollie


In [9]:
eng_pun_df = pd.read_csv("Enpunmix.csv",header=None,names=["English-Punjabi"])
eng_pun_df.head()

,English-Punjabi
0,ਪਰ ਆਮ ਤੌਰ 'ਤੇ ਇਸ ਵੱਲ attention ਨਹੀਂ ਦਿੱਤੀ ਜਾਂਦੀ.
1,"ਮੇਰੇ ਪਿਆਰੇ ਦੇਸ਼ ਵਾਸੀਓ, Mahatma Gandhi ਸਦਾ ਲਈ ਸ..."
2,ਤੁਹਾਡੇ ਫੈਸਲੇ ਅਜਿਹੇ benchmark ਤੇ ਅਧਾਰਤ ਹੋਣੇ ਚਾਹ...
3,ਸਾਨੂੰ stepwise ਤਰੀਕੇ ਨਾਲ ਉਨ੍ਹਾਂ ਦੀਆਂ ਮੁਸ਼ਕਲਾਂ ...
4,ਹੁਣ ਸਾਡੇ school ਵਿੱਚ ਲੜਕੀਆਂ ਦੇ ਨਾਲ ਨਾਲ ਮੁੰਡਿਆਂ...


In [10]:
for char in string.punctuation: 
    print(char, end=" ") 
    translate_table = dict((ord (char), None) for char in string.punctuation)

! " # $ % & ' ( ) * + , - . / : ; < = > ? @ [ \ ] ^ _ ` { | } ~ 

In [11]:
data_pun=[]
lang_pun=[]
for i,line in pun_df.iterrows(): 
    line = line ['Punjabi']
    if len(line) != 0:
        line = line.lower()
        line = re.sub(r"\d+", "", line)
        line = line.translate(translate_table)
        data_pun.append(line)
        lang_pun.append("Punjabi")

data_eng=[]
lang_eng=[]
for i,line in eng_df.iterrows(): 
    line = line ['English']
    if len(line) != 0:
        line = line.lower()
        line = re.sub(r"\d+", "", line)
        line = line.translate(translate_table)
        data_eng.append(line)
        lang_eng.append("English")

data_mix=[]
lang_mix=[]
for i,line in eng_pun_df.iterrows(): 
    line = line ['English-Punjabi']
    if len(line) != 0:
        line = line.lower()
        line = re.sub(r"\d+", "", line)
        line = line.translate(translate_table)
        data_mix.append(line)
        lang_mix.append("English-Punjabi")

In [12]:
df = pd.DataFrame({"Text": data_pun+data_eng+data_mix, "language":lang_pun+lang_eng+lang_mix})
print (df.shape)
df

(5921, 2)


,Text,language
0,ਉਨ੍ਹਾਂ ਨੇ ਸਾਰੇ ਤਜ਼ਰਬੇ ਵੱਖਰੀ ਕਿਸਮ ਦੇ ਕੀਤੇ ਹਨ,Punjabi
1,ਸ਼ਹਿਰਾਂ ਗਲੀਆਂ ਵਿਚ ਸਾਰੇ ਲਿਖਵਾ ਦਿਆਂ,Punjabi
2,ਆਊਟਡੋਰਸ ਚ ਪਏ ਟੋਇਆਂ ਦੀ ਰੀਪੇਅਰ ਕਰਵਾਓ,Punjabi
3,ਜੰਗ ਵਿਆਂ ਵਿੱਚ ਲੜੀ ਗਈ,Punjabi
4,ਧਰਮ ਦੇ ਨਾਂ ਤੇ ਦੇਸ਼ ਦੀ ਵੰਡ,Punjabi
...,...,...
5916,ਧੜਾਧੜ ਵੱਧ ਰਹੀ population ਤੇ ਵੀ ਰੋਕ ਸਖ਼ਤੀ ਨਾਲ ਲ...,English-Punjabi
5917,ਕਿਤਾਬਾਂ inspiration ਦਾ ਭੰਡਾਰ ਹਨ,English-Punjabi
5918,books ਪੜ੍ਹਨ ਨਾਲ life ਵਿਚ ਕੁਝ ਮਹਾਨ ਕਾਰਜ ਕਰਨ ਦੀ ...,English-Punjabi
5919,ਕਿਤਾਬਾਂ ਕਿਸੇ ਵੀ ਸੋਚ ਰਸਮ ਜਾਂ ਭਾਵਨਾ ਨੂੰ ਫੈਲਾਉਣ ਦ...,English-Punjabi


In [13]:
df_s = df.sample(frac = 1)
df_s

,Text,language
5407,ਇਹ ਪੈਸਾ march ਤੱਕ ਚੱਲੇਗਾ,English-Punjabi
1012,ਉਸੇ ਨੇ ਸਭ ਨੂੰ ਜੋੜ ਕੇ ਰੱਖਣਾ ਸੀ,Punjabi
5309,ਪਿਛਲੇ ten days ਵਿੱਚ ਕਾਫੀ ਹੰਗਾਮਾ ਹੋਇਆ ਹੈ।,English-Punjabi
3717,race organizers transport gear bins to design...,English
1220,ਸਾਡੀਆਂ ਵੀ ਗੱਲਾਂ ਗੂੜ੍ਹੀਆਂ ਹੋਣ ਲੱਗ ਪਈਆਂ,Punjabi
...,...,...
215,ਰਾਤ ਵੇਲੇ ਦੋਵੇਂ ਪਾਸੇ ਲਾਈਨਰ ਲਗਾਓ,Punjabi
2867,a lighthouse serves as a navigational aid for...,English
5916,ਧੜਾਧੜ ਵੱਧ ਰਹੀ population ਤੇ ਵੀ ਰੋਕ ਸਖ਼ਤੀ ਨਾਲ ਲ...,English-Punjabi
3949,ਸੈਰ ਸਪਾਟੇ ਨਾਲ ਜੁੜੀ information ਅਤੇ data ਦਾ ਆਦਾ...,English-Punjabi


In [14]:
X , Y= df_s.iloc[:,0],df_s.iloc[:,1]
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, random_state=0)
print (X_train.shape)
print (X_test.shape)
print (Y_train.shape)
print (Y_test.shape)

(4736,)
(1185,)
(4736,)
(1185,)


In [15]:
vectorizer=feature_extraction.text.TfidfVectorizer(ngram_range=(1,3),analyzer='char')
pipe_lr_r13=pipeline.Pipeline([
    ('vectorizer',vectorizer),
    ('clf',linear_model.LogisticRegression())
])

In [16]:
pipe_lr_r13.fit(X_train,Y_train)

Pipeline(steps=[('vectorizer',
                 TfidfVectorizer(analyzer='char', ngram_range=(1, 3))),
                ('clf', LogisticRegression())])

In [17]:
list(Y_test)

['English-Punjabi',
 'Punjabi',
 'English',
 'English',
 'Punjabi',
 'Punjabi',
 'English',
 'English-Punjabi',
 'English-Punjabi',
 'English-Punjabi',
 'Punjabi',
 'English',
 'Punjabi',
 'English-Punjabi',
 'English',
 'English',
 'English',
 'English',
 'English-Punjabi',
 'English',
 'English-Punjabi',
 'Punjabi',
 'English-Punjabi',
 'English',
 'English-Punjabi',
 'Punjabi',
 'English',
 'English-Punjabi',
 'Punjabi',
 'Punjabi',
 'Punjabi',
 'English-Punjabi',
 'Punjabi',
 'Punjabi',
 'Punjabi',
 'English-Punjabi',
 'Punjabi',
 'English-Punjabi',
 'English-Punjabi',
 'Punjabi',
 'English',
 'Punjabi',
 'English',
 'Punjabi',
 'English-Punjabi',
 'English',
 'Punjabi',
 'English',
 'English',
 'English-Punjabi',
 'Punjabi',
 'Punjabi',
 'Punjabi',
 'English',
 'Punjabi',
 'English',
 'English',
 'Punjabi',
 'English',
 'English-Punjabi',
 'Punjabi',
 'English',
 'English-Punjabi',
 'English',
 'Punjabi',
 'English-Punjabi',
 'English-Punjabi',
 'English',
 'English',
 'English-Pu

In [18]:
Y_predicted=pipe_lr_r13.predict(X_test)
list(Y_predicted)

['English-Punjabi',
 'Punjabi',
 'English',
 'English',
 'Punjabi',
 'Punjabi',
 'English',
 'English-Punjabi',
 'English-Punjabi',
 'English-Punjabi',
 'Punjabi',
 'English',
 'Punjabi',
 'English-Punjabi',
 'English',
 'English',
 'English',
 'English',
 'English-Punjabi',
 'English',
 'English-Punjabi',
 'Punjabi',
 'English-Punjabi',
 'English',
 'English-Punjabi',
 'Punjabi',
 'English',
 'English-Punjabi',
 'Punjabi',
 'Punjabi',
 'Punjabi',
 'English-Punjabi',
 'Punjabi',
 'Punjabi',
 'Punjabi',
 'English-Punjabi',
 'Punjabi',
 'English-Punjabi',
 'English-Punjabi',
 'Punjabi',
 'English',
 'Punjabi',
 'English',
 'Punjabi',
 'English-Punjabi',
 'English',
 'Punjabi',
 'English',
 'English',
 'English-Punjabi',
 'Punjabi',
 'Punjabi',
 'Punjabi',
 'English',
 'Punjabi',
 'English',
 'English',
 'Punjabi',
 'English',
 'English-Punjabi',
 'Punjabi',
 'English',
 'English-Punjabi',
 'English',
 'Punjabi',
 'English-Punjabi',
 'English-Punjabi',
 'English',
 'English',
 'English-Pu

In [19]:
combine = pd.DataFrame({'Predicted': Y_predicted,
                   'Expected': Y_test})
pd.set_option("display.max_rows", None, "display.max_columns", None)
print(combine)

            Predicted         Expected
4022  English-Punjabi  English-Punjabi
1042          Punjabi          Punjabi
2335          English          English
2343          English          English
1794          Punjabi          Punjabi
1137          Punjabi          Punjabi
2861          English          English
5697  English-Punjabi  English-Punjabi
4882  English-Punjabi  English-Punjabi
4402  English-Punjabi  English-Punjabi
251           Punjabi          Punjabi
3525          English          English
7             Punjabi          Punjabi
5308  English-Punjabi  English-Punjabi
3772          English          English
2713          English          English
3783          English          English
1895          English          English
5341  English-Punjabi  English-Punjabi
1890          English          English
5670  English-Punjabi  English-Punjabi
1593          Punjabi          Punjabi
5355  English-Punjabi  English-Punjabi
3877          English          English
5263  English-Punjabi  En

In [20]:
Y_predicted_values = []
for x in Y_predicted:
    if(x == 'Punjabi'):
        Y_predicted_values.append(0)
    elif(x == 'English'):
        Y_predicted_values.append(1)
    elif(x == 'English-Punjabi'):
        Y_predicted_values.append(2)
Y_predicted_values

[2,
 0,
 1,
 1,
 0,
 0,
 1,
 2,
 2,
 2,
 0,
 1,
 0,
 2,
 1,
 1,
 1,
 1,
 2,
 1,
 2,
 0,
 2,
 1,
 2,
 0,
 1,
 2,
 0,
 0,
 0,
 2,
 0,
 0,
 0,
 2,
 0,
 2,
 2,
 0,
 1,
 0,
 1,
 0,
 2,
 1,
 0,
 1,
 1,
 2,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 2,
 0,
 1,
 2,
 1,
 0,
 2,
 2,
 1,
 1,
 2,
 2,
 2,
 2,
 0,
 0,
 2,
 1,
 1,
 0,
 1,
 2,
 1,
 2,
 1,
 0,
 2,
 2,
 1,
 2,
 2,
 2,
 2,
 2,
 1,
 0,
 0,
 0,
 1,
 1,
 2,
 1,
 0,
 2,
 1,
 2,
 2,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 2,
 0,
 1,
 1,
 0,
 0,
 2,
 0,
 1,
 0,
 0,
 2,
 2,
 0,
 0,
 1,
 2,
 1,
 1,
 2,
 1,
 2,
 2,
 1,
 2,
 0,
 0,
 0,
 2,
 2,
 0,
 2,
 2,
 0,
 0,
 1,
 1,
 1,
 2,
 2,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 2,
 0,
 0,
 1,
 2,
 0,
 2,
 2,
 1,
 2,
 0,
 0,
 0,
 0,
 1,
 2,
 1,
 0,
 2,
 2,
 0,
 1,
 0,
 2,
 1,
 2,
 0,
 2,
 1,
 1,
 2,
 1,
 2,
 2,
 2,
 0,
 0,
 2,
 1,
 1,
 2,
 1,
 2,
 0,
 0,
 1,
 2,
 0,
 0,
 0,
 0,
 2,
 1,
 0,
 1,
 1,
 1,
 2,
 2,
 0,
 0,
 1,
 1,
 1,
 2,
 0,
 0,
 0,
 2,
 2,
 2,
 1,
 1,
 0,
 1,
 2,
 0,
 2,
 2,
 1,
 2,
 1,
 2,
 1,
 0,
 1,
 1,
 1,
 0,


In [21]:
acc=(metrics.accuracy_score(Y_test,Y_predicted))*100
acc

98.05907172995781

In [22]:
import pickle
lrfile = open('LRModel.pckl','wb')
pickle.dump(pipe_lr_r13,lrfile)
lrfile.close()

In [23]:
global lrLangDetectModel
lrLangDetectFile = open('LRModel.pckl', 'rb') 
lrLangDetectModel = pickle.load(lrLangDetectFile)
lrLangDetectFile.close()

In [24]:
def lang_detect (text):
    import numpy as np
    import string
    import re
    import pickle
    translate_table = dict((ord (char), None) for char in string.punctuation)
    global lrLangDetectModel
    lrLangDetectFile = open('LRModel.pckl', 'rb')
    lrLangDetectModel = pickle.load(lrLangDetectFile)
    lrLangDetectFile.close()
    text = "" .join(text.split())
    text = text.lower()
    text = re.sub (r"\d+", "", text)
    text = text.translate (translate_table)
    pred= lrLangDetectModel.predict ( [text])
    prob = lrLangDetectModel.predict_proba ([text])
    return pred[0]

In [25]:
lang_detect("Hello I am just building my own model")

'English'

In [26]:
lang_detect("ਤੁਹਾਨੂੰ ਮਿਲ ਕੇ ਬਹੁਤ ਖੁਸ਼ੀ ਹੋਈ")

'Punjabi'

In [27]:
lang_detect("ਸਿਹਤ ਅਤੇ ਪਰਿਵਾਰ ਭਲਾਈ ਵਿਭਾਗ ਅਧੀਨ ਖ਼ੁਦਮੁਖਤਿਆਰ ਸੰਸਥਾਵਾਂ ਨੂੰ ਦਲੀਲਪੂਰਨ ਬਣਾਉਣ ਵਿੱਚ ਅੰਤਰ ਮੰਤਰਾਲਾ, ਸਲਾਹਕਾਰ ਅਤੇ ਇਨ੍ਹਾਂ ਸੰਸਥਾਵਾਂ ਦੇ ਮੌਜੂਦਾ ਉਪ ਨਿਯਮਾਂ ਦੀ ਸਮੀਖਿਆ ਕਰਨਾ ਸ਼ਾਮਲ ਹੈ")

'Punjabi'

In [28]:
lang_detect("ਪ੍ਰਧਾਨ ਮੰਤਰੀ ਮੋਦੀ ਅਤੇ ਮਹਾਰਾਣੀ ਮੈਕਸਿਮਾ ਨੇ ਭਾਰਤ ਸਰਕਾਰ ਵੱਲੋਂ ਵਿੱਤੀ ਸਮਾਵੇਸ਼ ਵਿੱਚ ਵਾਧੇ ਲਈ ਪਿਛਲੇ ਕੁਝ ਵਰ੍ਹਿਆਂ ਵਿੱਚ ਜਨਧਨ ਯੋਜਨਾ, ਪ੍ਰਧਾਨ ਮੰਤਰੀ ਜੀਵਨ ਜਯੋਤੀ ਬੀਮਾ ਯੋਜਨਾ, ਪ੍ਰਧਾਨ ਮੰਤਰੀ ਬੀਮਾ ਸੁਰਕਸ਼ਾ ਯੋਜਨਾ ਅਤੇ ਅਟਲ ਪੈਂਸ਼ਨ ਯੋਜਨਾ ਜਿਹੇ ਚੁੱਕੇ ਗਏ ਵੱਖ-ਵੱਖ ਕਦਮਾਂ ‘ਤੇ ਵਿਚਾਰ-ਵਟਾਂਦਰਾ ਕੀਤਾ")

'Punjabi'

In [29]:
lang_detect("France reaffirmed its support for India’s candidature for a permanent membership of the UN Security Council")

'English'

In [30]:
lang_detect("hi my name is raghav jain")

'English'

In [31]:
lang_detect("ਹੈਲੋ ਮੇਰਾ ਨਾਮ ਰਾਘਵ ਜੈਨ ਹੈ")

'Punjabi'

In [32]:
lang_detect("Today is ਸ਼ੁੱਕਰਵਾਰ ਹੈ")

'English-Punjabi'